<a href="https://colab.research.google.com/github/jewetta/AAE590DSMM/blob/main/2pt_cor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, color

# Load grayscale image (e.g., TIFF, PNG)
image = io.imread("image_micro.png")
image = color.rgb2gray(image)
# Automatic thresholding using Otsu's method
thresh = filters.threshold_otsu(image)
binary = (image > thresh).astype(np.float32)


def two_point_direct(binary, r_max):
    height, width = binary.shape
    S2 = np.zeros(r_max)
    counts = np.zeros(r_max)
    phi1 = np.mean(binary)

    for i in range(height):
        for j in range(width):
            if binary[i, j] == 1:
                for di in range(-r_max, r_max + 1):
                    for dj in range(-r_max, r_max + 1):
                        r = int(np.hypot(di, dj))
                        if 0 < r < r_max:
                            i2 = (i + di) % height  # Periodic boundary
                            j2 = (j + dj) % width
                            if binary[i2, j2] == 1:
                                S2[r] += 1
                                counts[r] += 1

    return S2 / (counts * phi1**2 + 1e-12)


def radial_average(S2):
    L = S2.shape[0]
    y, x = np.indices((L, L)) - L // 2
    r = np.sqrt(x**2 + y**2).astype(int)

    r_flat = r.ravel()
    S2_flat = S2.ravel()

    r_bins = np.arange(0, np.max(r) + 1)
    radial_sum = np.bincount(r_flat, weights=S2_flat, minlength=len(r_bins))
    counts = np.bincount(r_flat, minlength=len(r_bins))

    return radial_sum / (counts + 1e-12)


# 1. Load and binarize
binary = ...  # From preprocessing

# 2. Compute 2-point correlation
S2 = two_point_direct(binary, r_max=50)

# 3. Radial average
r = np.arange(S2.shape[0] // 2)
S2_radial = radial_average(S2)[: len(r)]

# 4. Plot
plt.plot(r, S2_radial)
plt.xlabel('Radial Distance (pixels)')
plt.ylabel('S2(r)')
plt.title('Two-Point Correlation Function')
plt.grid(True)
plt.show()


AttributeError: 'ellipsis' object has no attribute 'shape'